In [3]:
import pandas as pd
import numpy as np

In [2]:
# Load the sales data from an Excel file without headers
# Loading without header is crucial because columns itself needs to be repaired
sales_data = pd.read_excel('https://foresightbi.com.ng/wp-content/uploads/2020/05/3.-Badly-Structured-Sales-Data-3.xlsx', header=None)

# Remove the last row, which contains summary or unwanted data
sales_data = sales_data.iloc[:-1]

sales_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,NaN,Ship Mode,First Class,NaN,NaN,Same Day,NaN,NaN,Second Class,NaN,NaN,Standard Class,NaN,NaN
1,NaN,Segment,Consumer,Corporate,Home Office,Consumer,Corporate,Home Office,Consumer,Corporate,Home Office,Consumer,Corporate,Home Office
2,Order ID,Order Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CA-2011-100293,2013-03-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.056
4,CA-2011-100706,2013-12-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,129.44,NaN,NaN,NaN,NaN,NaN


In [5]:
# Define the column names based on the shipping modes and their respective counts
# if you look at the first two rows in the excel file you will see that for each ship mode
# there are three segments Consumer, Corporate, and Home Office, therefore I replicated each
# of the ship mode thrice
shipping_modes = (
    ['First Class'] * 3 + 
    ['Same Day'] * 3 + 
    ['Second Class'] * 3 + 
    ['Standard Class'] * 3
)

shipping_modes[:10]

['First Class',
 'First Class',
 'First Class',
 'Same Day',
 'Same Day',
 'Same Day',
 'Second Class',
 'Second Class',
 'Second Class',
 'Standard Class']

In [6]:
# Set the first row with appropriate column names
sales_data.iloc[0] = [np.nan, 'Ship Mode'] + shipping_modes

# Transpose the DataFrame and skip the first two rows
transposed_data = sales_data.T.iloc[2:]

# Create a dictionary to hold the structured sales data
structured_sales_data = dict()

# Populate the dictionary with order details
for index, row in transposed_data.iterrows():
    non_na_values = row[row.notna()]
    structured_sales_data[(non_na_values[0], non_na_values[1])] = [
        (column_index, value) for column_index, value in zip(non_na_values.index[2:], non_na_values.values[2:])
    ]

# Initialize lists to store the extracted order details
order_ids = []
order_dates = []
shipping_modes = []
customer_segments = []
prices = []

# Extract order details from the structured sales data
for (shipping_mode, segment), values in structured_sales_data.items():
    for column_index, price in values:
        order_id, order_date = sales_data.iloc[column_index, :2]
        order_dates.append(order_date)
        order_ids.append(order_id)
        shipping_modes.append(shipping_mode)
        customer_segments.append(segment)
        prices.append(price)

# Create a new DataFrame with the structured order details
structured_order_data = pd.DataFrame({
    'Order ID': order_ids,
    'Order Date': order_dates,
    'Segment': customer_segments,
    'Ship Mode': shipping_modes,
    'Price': prices
})

# Display the first few rows of the structured order data
structured_order_data.head()


,Order ID,Order Date,Segment,Ship Mode,Price
0,CA-2011-103366,2013-01-15,Consumer,First Class,149.950
1,CA-2011-109043,2013-08-15,Consumer,First Class,243.600
2,CA-2011-113166,2013-12-24,Consumer,First Class,9.568
3,CA-2011-124023,2013-04-07,Consumer,First Class,8.960
4,CA-2011-130155,2013-05-19,Consumer,First Class,34.200
